Erzeuge aus den vorhanden Datenreihen 10 zusätzlich Datenreihen

In [10]:
import pandas as pd
import random
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

# Pfad zur CSV-Datei
file_path = 'csv_data/uberleitung_daten_76.csv'

# Lade die CSV-Datei
df_original = pd.read_csv(file_path)

# Kopie des DataFrames erstellen
df = df_original.copy()

# Datumsfelder identifizieren
date_columns = ['geb_datum', 'entlassung', 'datum_aktuell']

# Datumsfelder in datetime umwandeln
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%d.%m.%Y', errors='coerce')

# Erstelle das Metadaten-Objekt
metadata = SingleTableMetadata()

# Erkenne die Metadaten aus dem DataFrame
metadata.detect_from_dataframe(data=df)

# Liste der Spalten, die nicht als PII behandelt werden sollen
non_pii_columns = [
    'name', 'vorname', 'wohnort', 'krankenhaus', 'signature',
    'pflege_diagnose', 'einschraenkungen', 'besonderheit'
]

# Aktualisiere die Metadaten für diese Spalten
for column in non_pii_columns:
    metadata.update_column(
        column_name=column,
        sdtype='text',  # Setze den Datentyp auf 'text'
        pii=False       # Deaktiviere die PII-Erkennung
    )

# CTGAN-Synthesizer initialisieren
model = CTGANSynthesizer(
    metadata=metadata,
    epochs=500  # Anzahl der Trainings-Epochen
)

# Modell mit den vorhandenen Daten trainieren
model.fit(df)

# Anzahl der zu generierenden Datensätze
num_samples = 24

# Generiere neue synthetische Daten
synthetic_data = model.sample(num_samples)

# Datumsfelder wieder ins gewünschte Format bringen
for col in date_columns:
    synthetic_data[col] = pd.to_datetime(synthetic_data[col], errors='coerce')
    synthetic_data[col] = synthetic_data[col].dt.strftime('%d.%m.%Y')

# Spezifische Regeln anwenden
def apply_rules(row):
    # Regel für 'leistung' basierend auf 'entlassung_wohin'
    if row['entlassung_wohin'] == 'nach Hause':
        row['leistung'] = random.choice(['Sachleistung', 'Kombinationsleistung'])
    elif row['entlassung_wohin'] in ['Kurzzeitpflege', 'Pflegeeinrichtung']:
        row['leistung'] = 'Vollstationaer'
    else:
        row['leistung'] = row['leistung']
    
    # Regel für 'reha' basierend auf 'entlassung_wohin'
    if row['entlassung_wohin'] == 'Rehaklinik':
        row['reha'] = 'ja'
    else:
        row['reha'] = 'nein'
    
    # Regel für 'signature'
    row['signature'] = row['name']
    
    # Versichertennummer generieren
    row['versichertennummer'] = random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ') + ''.join(random.choices('0123456789', k=9))
    
    # Entlassung und aktuelles Datum setzen
    row['entlassung'] = '10.09.2024'
    row['datum_aktuell'] = '09.09.2024'
    
    return row

# Regeln auf die synthetischen Daten anwenden
synthetic_data = synthetic_data.apply(apply_rules, axis=1)

# Fehlende Werte auffüllen oder entfernen
synthetic_data.fillna('', inplace=True)

# Zusammenführen der Daten
complete_data = pd.concat([df_original, synthetic_data], ignore_index=True)

# Speichern in einer CSV-Datei
complete_data.to_csv('erweiterte_daten_neu.csv', index=False)

# Zeige die neuen synthetischen Daten
print(synthetic_data.head())

/Users/christiankolb/dev/synthetische_ueberleitungsantraege/myenv/lib/python3.12/site-packages/sdv/single_table/base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


      name  vorname   geb_datum geschlecht   wohnort pflegekasse  \
0   Müller     Hans  01.06.1932       mann  Augsburg         BKK   
1  Fischer  Herbert  29.03.1944       frau     Fürth         DAK   
2   Müller     Paul  19.04.1937       frau  Augsburg         AOK   
3   Wagner    Klara  18.12.1954       frau     Fürth          TK   
4    Maier    Klara  17.11.1942       mann  Augsburg         DAK   

  versichertennummer  entlassung   entlassung_wohin          krankenhaus  ...  \
0         W379234327  10.09.2024         nach Hause       Klinikum Fürth  ...   
1         C142278889  10.09.2024         nach Hause               Klinik  ...   
2         A731383882  10.09.2024         nach Hause  Klinikum Großhadern  ...   
3         K689055181  10.09.2024  Pflegeeinrichtung  Klinik Martha-Maria  ...   
4         C299864725  10.09.2024         nach Hause    Klinikum Augsburg  ...   

              information  mitteilen               verstehen  \
0  größtenteils vorhanden  vorhanden  gr

/var/folders/vj/kr8syxl515v8rt_mnrhw3m4m0000gn/T/ipykernel_88757/201297386.py:94: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  synthetic_data.fillna('', inplace=True)
